# Data Science Process using CRISP-DM

CRISP-DM merupakan salah satu proses yang sangat umum digunakan dalam data science. CRISP-DM, atau kepanjangan dari *Cross Industry Standard Process for Data Mining*, dalam data science, digunakan sebagai framework untuk memulai sebuah proyek data science sampai menemukan solusi yang dikehendaki. Ujung dari sebuah proyek data science terdiri dari 2 tujuan:
* untuk manusia → berupa laporan, presentasi, *insights*, dan sejenisnya
* untuk komputer → *deployment*, perangkat lunak, dan sejenisnya.

Secara umum, terdapat 8 tahapan dalam CRISP-DM:
1. *Business understanding*
2. *Data understanding*
3. *Data preparation*
4. *Modeling*
5. *Deployment*

Kita akan menggunakan data *[Customer Personality Analysis](https://www.kaggle.com/imakash3011/customer-personality-analysis)* untuk mempraktikkan CRISP-DM. Silakan unduh data melalui link tersebut dan simpan di dalam folder `data/marketing_campaign.csv`.

## A Look At The Data

Terkadang kita punya data yang akan kita pakai untuk mendefinisikan masalah yang akan kita selesaikan, itulah kenapa banyak perusahaan yang ingin menyimpan "terlebih dahulu" semua data yang berkaitan dengan perusahaan untuk kemudian dianalisis di lain waktu. Itu yang akan kita lakukan sekarang dan untuk memulai, kita impor beberapa library yang akan kita butuhkan.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.style.use("fivethirtyeight")

Kemudian, kita muat data `data/marketing_campaign.csv` menggunakan pemisah "tab" (`\t`).

In [ ]:
df = pd.read_csv("data/marketing_campaign.csv", sep="\t")
df.head()

### 1. Berapa jumlah baris dan kolom pada data?

Umumnya, setiap kali kita melakukan analisis data, kita tertarik untuk mencari tahu jumlah baris dan kolom pada data.

In [ ]:
num_rows = df.shape[0]
num_cols = df.shape[1]

print("# of rows:", num_rows)
print("# of columns:", num_cols)

### 2. *Data dictionary*

Setelah kita tahu jumlah kolom pada data, kita juga tertarik apa representasi dari masing-masing kolomnya, yang disebut dengan **data dictionary**. Untuk hal ini, biasanya seseorang dari tim bisnis, data engineer, data scientist, atau siapapun yang dulu menyimpan data tersebut, paham apa representasi dari tiap kolom. Pada kasus kita kali ini, berikut adalah *data dictionary* kita.


#### People

| Column Name | Representation |
| :---------- | :------------- |
| `ID` | Customer's unique identifier |
| `Year_Birth` | Customer's birth year |
| `Education` | Customer's education level |
| `Marital_Status` | Customer's marital status |
| `Income` | Customer's yearly household income |
| `Kidhome` | Number of children in customer's household |
| `Teenhome` | Number of teenagers in customer's household |
| `Dt_Customer` | Date of customer's enrollment with the company |
| `Recency` | Number of days since customer's last purchase |
| `Complain` | 1 if customer complained in the last 2 years, 0 otherwise |


#### Products

| Column Name | Representation |
| :---------- | :------------- |
| `MntWines` | Amount spent on wine in last 2 years |
| `MntFruits` | Amount spent on fruits in last 2 years |
| `MntMeatProducts` | Amount spent on meat in last 2 years |
| `MntFishProducts` | Amount spent on fish in last 2 years |
| `MntSweetProducts` | Amount spent on sweets in last 2 years |
| `MntGoldProds` | Amount spent on gold in last 2 years |


#### Promotion

| Column Name | Representation |
| :---------- | :------------- |
| `NumDealsPurchases` | Number of purchases made with a discount |
| `AcceptedCmp1` | 1 if customer accepted the offer in the 1st campaign, 0 otherwise |
| `AcceptedCmp2` | 1 if customer accepted the offer in the 2nd campaign, 0 otherwise |
| `AcceptedCmp3` | 1 if customer accepted the offer in the 3rd campaign, 0 otherwise |
| `AcceptedCmp4` | 1 if customer accepted the offer in the 4th campaign, 0 otherwise |
| `AcceptedCmp5` | 1 if customer accepted the offer in the 5th campaign, 0 otherwise |
| `Response` | 1 if customer accepted the offer in the last campaign, 0 otherwise |


#### Place

| Column Name | Representation |
| :---------- | :------------- |
| `NumWebPurchases` | Number of purchases made through the company’s web site |
| `NumCatalogPurchases` | Number of purchases made using a catalogue |
| `NumStorePurchases` | Number of purchases made directly in stores |
| `NumWebVisitsMonth` | Number of visits to company’s web site in the last month |


### 3. *Missing values*

Untuk setiap data yang akan kita analisis, nyatanya, akan sering mengandung beberapa nilai kosong atau yang disebut dengan *missing value*. Ada banyak hal yang menyebabkan adanya nilai kosong, seperti kesalahan pada sistem saat proses koleksi data atau responden yang tidak ingin mengisi kolom tersebut pada suatu survei. Oleh karena itu, kita perlu tahu keadaan ekstrim kolom, yaitu kolom mana saja yang tidak memiliki nilai kosong dan mana saja yang hampir semua nilainya kosong.

Kita bisa menggunakan metode `isna` pada objek DataFrame.

```python
df.isna()
```

In [ ]:
cols_no_na = df.columns[df.isna().sum(axis=0) == 0]
cols_with_na = df.columns[df.isna().sum(axis=0) > 0]

print("Columns with no missing value:", cols_no_na)
print("Columns with missing values:", cols_with_na)

```{admonition} Eksplorasi
Coba hitung persentase *missing value* dari tiap kolom!
```

In [ ]:
# KETIK DI SINI


Selain kita tinjau tiap kolom, kita mungkin juga perlu melihat *missing value* untuk tiap baris data, apakah ada baris yang lebih dari 50% atributnya merupakan nilai kosong. Berbeda dengan kolom, untuk meninjau baris, kita gunakan `axis=1`.

```python
df.isna().sum(axis=1)
```

In [ ]:
df[df.isna().sum(axis=1) > 0]

```{tip}
Pandas menyediakan metode `DataFrame.info()` yang akan menampilkan informasi terkait tipe data pada setiap kolom dan jumlah *non-missing values*.
```

In [ ]:
df.info()

### 4. `Dtype` Tiap Kolom

Jika kita lihat hasil dari metode `info` dan dicocokkan dengan *data dictionary*, kolom `Dt_Customer` sepertinya merepresentasikan sebuah tanggal tapi tipe datanya masih `object`. Oleh karena itu, kita perlu mengubah tipe data `Dt_Customer` menjadi `datetime`.

```{note}
Pandas memiliki fitur yang akan sangat membantu kita untuk berinteraksi dengan data `datetime`. Oleh karena itu, sangat dianjurkan semua tipe data yang mewakili data tanggal/waktu diubah menjadi `datetime`.
```

````{panels}
:column: col-sm


**Convert to datetime with `astype`**
^^^
`astype` dapat digunakan pada `Series` ataupun `DataFrame`.

```python
Series.astype("datetime64[ns, UTC]")
```
---

**Convert to datetime with `pd.to_datetime`**
^^^
Menggunakan fungsi bawaan pandas:

```python
pandas.to_datetime(Series)
```
````

In [ ]:
df.Dt_Customer = df.Dt_Customer.astype("datetime64")

### 5. Visualisasi Sederhana

Saatnya sekarang kita membuat visualisasi awal yang sederhana untuk mencari tahu karakter dari masing-masing kolom. Jenis visualisasi yang akan kita buat biasanya berupa *univariate* ataupun *multivariate* (mungkin maksimal 3-4 variabel).

```{admonition} Bar Plot
:class: warning
Bagaimana distribusi status kawin dari setiap customer?
```

In [ ]:
outlier_year = df.Year_Birth < 1940
outlier_income = df.Income > 200000

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x="Marital_Status", data=df)
plt.title("Kebanyakan Customer Berstatus Kawin")
plt.xlabel("Status Kawin", loc="left")
plt.ylabel("Count", loc="top")
plt.grid(False)
plt.show()

```{admonition} Scatter Plot
:class: warning
Bagaimana hubungan antara umur dan gaji?
```

In [ ]:
plt.figure(dpi=150)
sns.scatterplot(x="Year_Birth", y="Income", data=df[~outlier_year & ~outlier_income], color="gray")
sns.scatterplot(x="Year_Birth", y="Income", data=df[outlier_year | outlier_income], color="royalblue")
plt.xlabel("Tahun Kelahiran", loc="left")
plt.ylabel("Income", loc="top")
plt.title(f"4 Customer Yang Berbeda Dengan Lainnya", loc="left")
plt.text(1980, 650000, "Outlier", color="royalblue")
plt.text(1900, 120000, "Outlier", color="royalblue", horizontalalignment="center")
plt.grid(False)
plt.show()

Dari gambar di atas, terlihat ada setidaknya **4 customer** yang berbeda dengan yang lain. Dalam visualisasi, *outlier* ini memaksa skala dalam grafik untuk menampung semua titik, sehingga kita kurang bisa melihat kebanyakan data. Oleh karena itu, kita bisa buat visualisasi tanpa keempat *outlier*.

In [ ]:
avg_year = df.Year_Birth.mean().round()
avg_income = df.Income.mean().round(2)

In [ ]:
plt.figure(dpi=150)
sns.scatterplot(x="Year_Birth", y="Income", data=df[~outlier_income & ~outlier_year], color="gray")
sns.regplot(x="Year_Birth", y="Income", data=df[~outlier_income & ~outlier_year], scatter=False, line_kws=dict(linewidth=2), color="royalblue")
plt.xlabel("Tahun Kelahiran", loc="left")
plt.ylabel("Income", loc="top")
plt.title("Korelasi Negatif yang Lemah Antara Birth Year - Income", loc="left")
plt.grid(False)
plt.show()

In [ ]:
customer_by_date = df.set_index("Dt_Customer").resample("D")
customer_by_month = df.set_index("Dt_Customer").resample("M")
customer_by_year = df.set_index("Dt_Customer").resample("Y")
customer_by_quarter = df.set_index("Dt_Customer").resample("Q")

plt.figure(figsize=(15, 6), dpi=80)
ax = sns.lineplot(
    x=customer_by_quarter.count().index.date,
    y="ID",
    data=customer_by_quarter.count(),
    marker="o",
    color="gray",
    size=14
)
plt.xticks(
#     range(len(customer_by_quarter.groups)),
    customer_by_quarter.count().index.date,
    [d.month_name()[:3] + " '" + str(d.year)[-2:] for d in customer_by_quarter.groups],
    fontsize=12
)
plt.yticks(ax.get_yticks(), fontsize=12)
plt.legend([])
plt.xlabel("Join Date", loc="left")
plt.ylabel("# of Customers", loc="top")
plt.title("Jumlah Penambahan Customer Terbanyak di Q1 dan Q4 2013", loc="left", y=1.1)
plt.grid(False)
plt.show()

```{admonition} Apa Saja Yang Perlu Kita Cek
:class: tip
✅ Cari tahu jumlah baris dan kolom pada data

✅ Cari tahu apakah ada *missing value* pada masing-masing baris dan/atau kolom

✅ Cek apakah kolom merepresentasikan tipe data yang seharusnya (contoh: *numeric* yang dianggap *string*)

✅ Buat visualisasi eksplorasi seperti barplot, boxplot, scatterplot, dan lainnya
```


## Business Questions

Setelah kita melihat-lihat data yang kita gunakan, sekarang saatnya untuk memformulasikan masalah yang ingin diselesaikan atau informasi apa yang kita gali dari data.

````{note}
Terkadang kita memulai dengan keadaan sudah memiliki data, sehingga kita perlu mencari tahu informasi apa yang ingin digali. Tapi, bisa juga kita sudah mempunyai masalah bisnis yang ingin kita pecahkan atau cari tahu, tapi kita belum menemukan data yang cocok seperti apa.

```{figure} ./assets/images/data-question-solution.png
:alt: Data - Question - Solution
:name: data-question-solution
:width: 50%
Data - Question - Solution
```

```{figure} ./assets/images/question-data-solution.png
:alt: Question - Data - Solution
:name: question-data-solution
:width: 50%
Question - Data - Solution
```
````

Setelah melihat secara singkat dan membuat visualisasi data, kamu bisa membuat beberapa pertanyaan bisnis yang menurut kamu menarik untuk dicari. Sebagai contoh, berikut beberapa pertanyaan bisnis yang bisa kita eksplor sama-sama:

1. Bagaimana `Education` berhubungan dengan `Income` yang mungkin juga berhubungan dengan **jumlah total pembelian**?
2. *Customer* seperti apakah yang sering membeli produk melalui Web, Toko, serta Katalog produk?


## Data Understanding

Dari ketiga *business problem* tersebut, kita terjemahkan ke dalam data bagaimana **asumsi dan langkah awal** untuk menjawab pertanyaan tersebut. Mungkin kita akan memerlukan *modeling* menggunakan *machine learning*, tapi setidaknya kita harus mencoba menjawab pertanyaan tersebut menggunakan data yang tersedia dulu.


### 1. Hubungan `Education` dengan `Income` dengan Jumlah Total Pembelian

Pertama, kita harus membuat kolom baru yang menjumlahkan semua pembelian, misal kita namai `NumTotalPurchases`.

In [ ]:
df["NumTotalPurchases"] = df["NumCatalogPurchases"] + df["NumDealsPurchases"] + df["NumWebPurchases"]

Jika kita lihat bagaimana hubungan antara `Income` dengan `NumTotalPurchases` di bawah ini, sekilas memang terdapat relasi positif diantara keduanya. Tapi, mari kita coba lihat lebih jauh lagi.

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)
sns.scatterplot(x="Income", y="NumTotalPurchases", data=df, color="gray")
sns.regplot(
    x="Income", y="NumTotalPurchases", data=df, scatter=False, line_kws=dict(linewidth=2), color="royalblue"
)
plt.xlabel("Income", loc="left")
plt.ylabel("Total Purchases", loc="top")
plt.title("Strong Positive Corrletion Between Income and Total Purchases", loc="left")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), dpi=80)
sns.boxplot(
    x="Income",
    y="Education",
    data=df
)
plt.xlabel("Income", loc="left")
plt.xticks(size=10)
plt.ylabel("Eduation", loc="top")
plt.title("Harder To Look Due To Outliers", loc="left")
plt.show()

Dari gambar di atas, kita terasa sulit untuk melihat secara teliti bagaimana sebaran `Income` pada masing-masing jenis `Education` karena beberapa outliers. Oleh karena itu, untuk melihat lebih dalam lagi, kita perlu menghilangkan outlier tersebut untuk "sementara". Kita bisa gunakan kode di bawah untuk menghilangkan beberapa outliers pada `Income`.

```python
df_no_outliers = df[df.Income < df.Income.quantile(.75) + 1.5*(df.Income.quantile(.75) - df.Income.quantile(.25))]

plt.figure(figsize=(10, 6), dpi=80)
sns.boxplot(
    x="Income",
    y="Education",
    data=df_no_outliers
)
plt.xlabel("Income", loc="left")
plt.xticks(size=10)
plt.ylabel("Eduation", loc="top")
plt.title("")
plt.show()
```

In [ ]:
df_no_outliers = df[
    df.Income < df.Income.quantile(.75) + 1.5*(df.Income.quantile(.75) - df.Income.quantile(.25))
]

plt.figure(figsize=(10, 6), dpi=80)
sns.boxplot(
    x="Income",
    y="Education",
    data=df_no_outliers,
    showmeans=True, meanprops={"markerfacecolor": "white", "marker": "o"},
    order=["PhD", "Master", "Graduation", "Basic", "2n Cycle"]
)
plt.xlabel("Income", loc="left")
plt.xticks(size=10)
plt.ylabel("Eduation", loc="top")
plt.title(
    "PhD Customers Have The Highest Average Income, \nGraduation Customers Have The Highest Variance",
    loc="left"
)
plt.show()

Dari gambar di atas, terlihat bahwa customer dengan latar belakang edukasi PhD memiliki rata-rata pendapatan yang paling tinggi dibanding lainnya, sedangkan pendapatan dengan latar belakang edukasi *Graduation* paling bervariasi dari pada yang lain. Sedangkan, *Basic* memiliki rata-rata dan variansi pendapatan yang paling rendah.

Selanjutnya, kita coba lihat bagaimana hubungan antara pendapatan pada masing-masing edukasi terhadap jumlah total pembelian.

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(10, 8), sharex=True, sharey=True)
for axis, edu in zip(ax, ["PhD", "Graduation", "Master", "Basic"]):
    sns.scatterplot(
        x="Income",
        y="NumTotalPurchases",
        data=df_no_outliers[df_no_outliers.Education == edu],
        ax=axis,
        alpha=.5,
        color="gray"
    )
    sns.regplot(
        x="Income",
        y="NumTotalPurchases",
        data=df_no_outliers[df_no_outliers.Education == edu],
        ax=axis,
        scatter=False,
        line_kws=dict(linewidth=2), color="royalblue"
    )
    axis.annotate(
        "Education="+ edu,
        xycoords="axes fraction",
        xy=(1.02, .5),
        bbox={"boxstyle": "round", "fc": "w", "alpha": .3}
    )
    axis.set_ylabel("")
fig.text(.02, .9, "Total Purchases", rotation="vertical", va="top", fontsize=18)
plt.xlabel("Income", loc="left", fontsize=18)
plt.suptitle("PhD, Graduation, & Master Education Income Correlate Positively", x=0.5, y=.95)
plt.show()

Berdasarkan visualisasi data di atas, pendapatan customer dengan latar belakang edukasi *PhD*, *Graduation*, *Master* memiliki korelasi positif terhadap jumlah pembelian yang dilakukan, meskipun tidak sebegitu kuat dibanding pendapatan secara keseluruhan.

### 2. Customer Yang Membeli Melalui Web, Toko, dan Katalog



In [ ]:
cust_no_purchases = df[df["NumTotalPurchases"] == 0]
